In [ ]:
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from tensorflow.keras.layers import Conv1D, MaxPooling1D, AveragePooling1D
from tensorflow.keras.utils import to_categorical

In [ ]:
# drive path
drive = '/content/drive/My Drive/Colab Notebooks/HAR'

# local path
local = 'G:/My Drive/Colab Notebooks/HAR'

hapt = '/HAPT Data Set'       # Human Activity Postural Transitions data set
har = '/UCI HAR Dataset'      # Human Activity Recognition data set

path_drive = drive + har
path_local = local + har

In [ ]:
using_drive = 1

if using_drive == 1:
  from google.colab import drive
  drive.mount('/content/drive')
  path = path_drive
else:
  path = path_local

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Gravitational acceleration data files for x, y and z axes: `total_acc_x_train.txt`, `total_acc_y_train.txt` , `total_acc_z_train.txt`.<br>
Body acceleration data files for x, y and z axes: `body_acc_x_train.txt`, `body_acc_y_train.txt`, `body_acc_z_train.txt`.<br>
Body gyroscope data files for x, y and z axes: `body_gyro_x_train.txt`,
`body_gyro_y_train.txt`, `body_gyro_z_train.txt`.<br>
The structure is mirrored in the test directory.

In [ ]:
# load a single file as a numpy array
def load_file(filepath):
  dataframe = read_csv(filepath, header=None, delim_whitespace=True, encoding='utf-8', encoding_errors='ignore')
  return dataframe.values

# load a list of files, such as x, y, z data for a given variable
def load_group(filenames, prefix=''):
  loaded = list()
  for name in filenames:
    data = load_file(prefix + name)
    loaded.append(data)
  # stack group so that features are the 3rd dimension
  loaded = dstack(loaded)
  return loaded

# load a dataset group, such as train or test
def load_dataset_group(group, prefix=''):
  filepath = prefix + '/' + group + '/Inertial Signals/'
  # load all 9 files as a single array
  filenames = list()
  # total acceleration
  filenames += ['total_acc_x_'+group+'.txt', 'total_acc_y_'+group+'.txt', 'total_acc_z_'+group+'.txt']
  # body acceleration
  filenames += ['body_acc_x_'+group+'.txt', 'body_acc_y_'+group+'.txt', 'body_acc_z_'+group+'.txt']
  # body gyroscope
  filenames += ['body_gyro_x_'+group+'.txt', 'body_gyro_y_'+group+'.txt', 'body_gyro_z_'+group+'.txt']
  # load input data
  X = load_group(filenames, filepath)
  # load class output
  # y = load_file(prefix + '/' + group + '/y_' + group + '.txt') # doesn't work with default 'utf-8'
  y = read_csv(prefix + '/' + group + '/y_' + group + '.txt', header=None, encoding='utf-16')
  return X, y

# load the dataset, returns train and test X and y elements
def load_dataset(path=''):
  # print(path)
  # load all train
  trainX, trainy = load_dataset_group('train', path)
  # load all test
  testX, testy = load_dataset_group('test', path)
  # zero-offset class values
  trainy = trainy - 1
  testy = testy - 1
  # one hot encode y
  trainy = to_categorical(trainy)
  testy = to_categorical(testy)
  return trainX, trainy, testX, testy

In [ ]:
# fit and evaluate a model
def evaluate_model(trainX, trainy, testX, testy):
  verbose, epochs, batch_size = 1, 10, 32
  n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
  model = Sequential()
  model.add(Conv1D(64, 3, activation='relu', input_shape=(n_timesteps,n_features))) # filters=64, kernel_size=3 (length of the 1D convolution window)
  model.add(Conv1D(64, 3, activation='relu'))
  model.add(Dropout(0.5))
  model.add(AveragePooling1D()) # pool_size=2, strides=None, padding="valid" (no padding), or "same" (results in padding evenly to the left/right or up/down of the input such that output has the same height/width dimension as the input)
  model.add(Flatten())
  model.add(Dense(100, activation='relu'))
  model.add(Dense(n_outputs, activation='softmax'))
  opt = keras.optimizers.Adam(learning_rate=0.01) # default 0.001
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  # fit network
  model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=0)
  # evaluate model
  _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=verbose)
  return accuracy, model

In [ ]:
# summarize scores
def summarize_results(scores):
  print(scores)
  m, s = mean(scores), std(scores)
  print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))

In [ ]:
# run an experiment
def run_experiment(repeats=10, path=''):
  # load data
  trainX, trainy, testX, testy = load_dataset(path)
  # repeat experiment
  scores = list()
  for r in range(repeats):
    score, model = evaluate_model(trainX, trainy, testX, testy)
    score = score * 100.0
    print('>#%d: %.3f' % (r+1, score))
    scores.append(score)
  # summarize results
  summarize_results(scores)
  print(model.summary())

In [ ]:
# run the experiment
run_experiment(path=path)

/content/drive/My Drive/Colab Notebooks/HAR/UCI HAR Dataset
93/93 [==============================] - 0s 3ms/step - loss: 0.3728 - accuracy: 0.9121
>#1: 91.211
93/93 [==============================] - 0s 3ms/step - loss: 0.5350 - accuracy: 0.9138
>#2: 91.381
93/93 [==============================] - 0s 3ms/step - loss: 0.6244 - accuracy: 0.9111
>#3: 91.110
93/93 [==============================] - 0s 3ms/step - loss: 0.3361 - accuracy: 0.9209
>#4: 92.094
93/93 [==============================] - 1s 4ms/step - loss: 0.6163 - accuracy: 0.9192
>#5: 91.924
93/93 [==============================] - 0s 3ms/step - loss: 0.3933 - accuracy: 0.9135
>#6: 91.347
93/93 [==============================] - 0s 3ms/step - loss: 0.5274 - accuracy: 0.9046
>#7: 90.465
93/93 [==============================] - 1s 4ms/step - loss: 0.7446 - accuracy: 0.8985
>#8: 89.854
93/93 [==============================] - 1s 4ms/step - loss: 0.4603 - accuracy: 0.8972
>#9: 89.718
93/93 [==============================] - 1s 4ms/s